# Making Audio Signals with Constant Phase Shifts

In [ ]:
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
import util
from os.path import join
from scipy.signal import fftconvolve as conv, kaiser, freqz, hann, butter, lfilter
%matplotlib inline

import importlib
importlib.reload(util)

## Constant Phase Shifters

* Frequency independent phase shift: $\phi = -180^\circ,(15^\circ),180^\circ$
* Sampling rate: 44.1 kHz
* Filter order (length): 4096 (8193)
* Blackman window (`scipy.signal.kaiser` with `beta=8.6`)

In [ ]:
fs = 44100
reference_phase = 0
phimin, phimax, dphi = -180, 180, 15
phase_angles_deg = np.arange(phimin, phimax + dphi, dphi)
phase_angles = np.pi / 180 * phase_angles_deg
filter_order = 2**16
half_length = filter_order / 2
group_delay = half_length / fs
beta = 8.6

_, href = util.constant_phase_shifter(filter_order, reference_phase, beta=beta)
phase_shifters = [util.constant_phase_shifter(filter_order, phi, beta=beta)[1]
                  for phi in phase_angles]

### Frequency Responses

In [ ]:
from matplotlib import cm
colors = cm.viridis.colors
viridis = cm.get_cmap('viridis', len(phase_angles)).colors

fmin, fmax, fnum = 0.1, fs / 2, 500
f = np.logspace(np.log10(fmin), np.log10(fmax), num=fnum)
omega = 2 * np.pi * f

fig, axes = plt.subplots(figsize=(10, 10), ncols=2, gridspec_kw={'wspace':0.05})
voffset = 5

for i, (phi, h) in enumerate(zip(phase_angles_deg, phase_shifters)):
    _, H = freqz(h, 1, f, fs=fs)
    H *= np.exp(1j * omega * group_delay)
    axes[0].semilogx(f, util.db(H) -i * voffset, c=viridis[i], alpha=0.75)
    axes[1].semilogx(f, np.unwrap(np.angle(H)) / np.pi, c=viridis[i], alpha=0.75)
    axes[0].text(1000, -i * voffset, '{:0.0f}'.format(phi))
axes[0].set_ylim(-voffset * len(phase_angles), 3)
axes[0].set_ylabel('$|H(\omega)|$ / dB')
axes[0].set_title('Magnitude')
axes[1].yaxis.tick_right()
axes[1].yaxis.set_label_position("right")
axes[1].set_ylim(-1.1, 1.1)
axes[1].set_ylabel(r'$\angle H(\omega)$ $/\pi$')
axes[1].set_title('Phase')

for ax in axes:
    ax.set_xlim(fmin, fmax)
    ax.set_xlabel('$f$ / Hz')
    ax.grid()

Note the deviations in magnitude and phase responses for $\varphi \neq 0, \pi$.
The distortion in the magnitude spectrum is most pronounced for $\varphi = \pm\frac{\pi}{2}$,
whereas the phase distortion exhibits a slightly different dependency.
At $\omega = 0$, the phase tends to 0 for $\varphi \in (-\frac{\pi}{2}, \frac{\pi}{2})$,
and $\pi$ for $\varphi \in (-\pi, -\frac{\pi}{2}) \cup (\frac{\pi}{2}, \pi)$.
The phase responses are accurate for $\varphi=0, \pm\frac{\pi}{2}, \pi$
throughout the entire frequency range.

### Audibility of Low Frequency Attenuation

For a practical implementation of constant phase shifter,
the impulse response have to be truncated to a finite length
and possibly be windowed in order to smooth out the spectral fluctuations
(whether it improves the perceptual quality is unclear though).
As demonstrated above, there is a trade off between
the length of the FIR coefficients and the spectral distortion
around the base band and the Nyquist frequency.
Therefore, it is of interest to what extent
the magnitude distortion is audible for varying FIR lengths.

In order to examine the perceptual influence of the truncation,
constant phase shifters of different lengths are built,

$$N = 2^{\{4,\ldots,20\}}.$$

The FIR filters are then applied to a selected input signal.

Considering the frequency dependent sensitivity
and resolution of human hearing ability,
the low frequency distortion is likely to be detected more easily.
Moreover, frequency components around the Nyquist frequency
have less practical relevance.
The audibility of the low frequency distortions is thus
of primary interest in designing FIR constant phase shifters.

In the following example, a pink noise train is used as the stimulus.

The phase angle of $\varphi=\frac{\pi}{2}$ is chosen
which is the worst case scenario in terms of magnitude distortion.
Since no phase distortion occurs for this phase angle,
it is suited to examine only the influence
of the attenuation of the magnitude spectrum.

In [ ]:
# Pulse train of raised cosine pink noise

fs = 44100
repetitions = 3
pulse_length = 500  # in milliseconds
silence = 100, 50, 100  # pre-, inter-, post-
pink_train = util.pink_train(pulse_length, repetitions, silence, fs)
t = np.arange(len(pink_train)) / fs * 1000

# Optional low-pass filtering
# b, a = butter(4, 0.1)
# pink_train = lfilter(b, a, pink_train)
# pink_train *= 0.9 / np.max(np.abs(pink_train))

plt.plot(t, pink_train)
plt.xlabel('$t$ / ms')
plt.grid()

In [ ]:
out_dir = '../data/stimuli'
# out_dir = '../abx_software/webMUSHRA_c929877_20180814/configs/resources/stimuli'
suffix = '.wav'

filter_order_exponents = np.arange(4, 21)  # {2^4, ..., 2^20}
beta = 8.6
phase_angle = np.pi / 2
fade_in, fade_out = 100, 100

for m in filter_order_exponents:
    filter_order = 2**m
    half_length = int(filter_order / 2)
    h = util.constant_phase_shifter(filter_order, phase_angle, beta=beta)[1]
    y = conv(pink_train, h)[half_length:-half_length]
    y = util.fade(y, fade_in, fade_out, 'h')
    data_name = 'pink_train_phi{:0.0f}_m{:.0f}'.format(np.rad2deg(phase_angle), m)
    if (np.abs(y) > 1).any():
        print('Warning: The amplitude ({:0.2f}) exceeds 1 in {}'
              .format(np.max(np.abs(y)), data_name))
    sf.write(join(out_dir, data_name + suffix), y, fs ,subtype='PCM_24')

### Recursive High Pass Filter

One possible way to avoid any unwanted effects
by the low frequency deviations is to high pass filter the stimuli.

In [ ]:
hpf_order = 2
f_cutoff = 12.5
b, a = butter(hpf_order, Wn=f_cutoff, btype='high', fs=fs)
_, H_hpf = freqz(b, a, worN=f, fs=fs)

plt.figure()
plt.semilogx(f, util.db(H_hpf))
plt.plot(f_cutoff, 0, 'v')
plt.xlabel('$f$ / Hz')
plt.ylabel('Magnitude / dB')
plt.grid()

### Linear Phase High Pass Filter

Alternatively, the reference stimuli (zero phase shift)
can be equalized with a linear phase FIR filter
which exhibits the same magnitude response
as the constant phase shifter.

In [ ]:
filter_order = 2**6
filter_length = filter_order + 1
half_length = int(filter_order / 2)
phase_angle = -np.pi / 2
_, h = util.constant_phase_shifter(filter_order, phase_angle, beta=beta)

nfft = filter_length

dc_offset = 0 / nfft

h_eq = np.fft.irfft(np.abs(np.fft.rfft(h, n=nfft)), n=nfft)
h_eq = np.roll(h_eq, half_length)

h -= dc_offset
h_eq -= dc_offset

fmin, fmax, fnum = 0.1, fs / 2, 5000
f = np.logspace(np.log10(fmin), np.log10(fmax), num=fnum)
_, H = freqz(h, 1, worN=f, fs=fs)
_, H_eq = freqz(h_eq, 1, worN=f, fs=fs)

f_dft = fs * np.arange(half_length + 1) / nfft
H_dft = np.fft.rfft(h)
H_dft_eq = np.fft.rfft(h_eq)

fig, ax = plt.subplots(figsize=(15, 6), ncols=2)

ax[0].plot(h, label='Phase shifter')
ax[0].plot(h_eq, label='Equalizer')
ax[0].legend()
ax[0].grid()

# ax[1].semilogx(f, util.db(H), 'b')
# ax[1].semilogx(f, util.db(H_eq), 'r')
# ax[1].semilogx(f_dft, util.db(H_dft), 'bo')
# ax[1].semilogx(f_dft, util.db(H_dft_eq), 'rx')
ax[1].plot(f, util.db(H), 'b')
ax[1].plot(f, util.db(H_eq), 'r')
ax[1].plot(f_dft, util.db(H_dft), ls='none', marker='o', mfc='none', mec='b')
ax[1].plot(f_dft, util.db(H_dft_eq), 'rx')
ax[1].set_xlabel('$f$ / Hz')
ax[1].set_ylabel('Magnitude / dB')
ax[1].set_xlim(1, 3000)
# ax[1].set_ylim(-100)
ax[1].grid()

## Make Stimuli

* Source signals ([`data/source-signals`](../data/source-signals)): speech, castanets, and pink noise
* Phase shifted signals will be saved in [`data/stimuli`](../data/stimuli)
* Diotic: phase shift on both signals
* Dichotic: phase shift either on left or right signal


In [ ]:
src_dir = '../data/source-signals'
out_dir = '../data/stimuli'
# out_dir = '../abx_software/webMUSHRA_c929877_20180814/configs/resources/stimuli'
suffix = '.wav'
file_names = ['castanets']
fade_in, fade_out = 32, 32

filter_order = 2**21
half_length = int(filter_order / 2)
href = util.constant_phase_shifter(filter_order, reference_phase, beta=beta)[1]
phase_shifters = [util.constant_phase_shifter(filter_order, phi, beta=beta)[1]
                  for phi in phase_angles]

for fn in file_names:
    sig, fs = sf.read(join(src_dir, fn + suffix))
#     sig = lfilter(b, a, sig)  # optional HPF
    crest_factors = np.zeros_like(phase_angles)
    
    if fn == 'pink_noise':  # hacked :-(
        sig = 10**(-3/20) * sig
    
    sig_ref = util.fade(conv(sig, href)[half_length:-half_length], fade_in, fade_out)
    sf.write(join(out_dir, fn + '_ref' + suffix),
             np.column_stack((sig_ref, sig_ref)), fs, subtype='PCM_24')
    for i, (phi, h) in enumerate(zip(phase_angles_deg, phase_shifters)):
        sig_shift = util.fade(conv(sig, h)[half_length:-half_length], fade_in, fade_out)
        crest_factors[i] = util.crest_factor(sig_shift, oversample=4)
        
        if np.amax(np.abs(sig_shift)) >= 1:
            print('sig_shift will clip at:', fn, phi, 'simple peak=', np.amax(np.abs(sig_shift)))

        sf.write(join(out_dir, fn + '_phi{}'.format(phi) + suffix),
                 np.column_stack((sig_shift, sig_shift)), fs, subtype='PCM_24')
    np.savetxt(join(out_dir, fn + '_crestfactor.txt'), np.stack([phase_angles, crest_factors]))

The crest factors of the stimuli are saved in the same directory as a text file.
It can be loaded as following.

In [ ]:
ph, cf = np.loadtxt(join(out_dir, file_names[0] + '_crestfactor.txt'))

plt.plot(np.rad2deg(ph), util.db(cf), '-o')
plt.xlabel(r'$\varphi$ / deg')
plt.ylabel('Crest Factor / dB')
plt.grid()

### Audition

#### Diotic

| $\phi$ | speech | Castanets | Pink Noise |
|:------:|:------:|-----------|:----------:|
| Ref. | <audio type="audio/wave" src="../data/stimuli/speech_ref.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_ref.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_ref.wav" controls></audio> |
| -180 | <audio type="audio/wave" src="../data/stimuli/speech_phi-180_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi-180_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi-180_diotic.wav" controls></audio> |
| -165 | <audio type="audio/wave" src="../data/stimuli/speech_phi-165_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi-165_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi-165_diotic.wav" controls></audio> |
| -150 | <audio type="audio/wave" src="../data/stimuli/speech_phi-150_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi-150_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi-150_diotic.wav" controls></audio> |
| -135 | <audio type="audio/wave" src="../data/stimuli/speech_phi-135_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi-135_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi-135_diotic.wav" controls></audio> |
| -120 | <audio type="audio/wave" src="../data/stimuli/speech_phi-120_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi-120_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi-120_diotic.wav" controls></audio> |
| -105 | <audio type="audio/wave" src="../data/stimuli/speech_phi-105_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi-105_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi-105_diotic.wav" controls></audio> |
| -90 | <audio type="audio/wave" src="../data/stimuli/speech_phi-90_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi-90_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi-90_diotic.wav" controls></audio> |
| -75 | <audio type="audio/wave" src="../data/stimuli/speech_phi-75_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi-75_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi-75_diotic.wav" controls></audio> |
| -60 | <audio type="audio/wave" src="../data/stimuli/speech_phi-60_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi-60_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi-60_diotic.wav" controls></audio> |
| -45 | <audio type="audio/wave" src="../data/stimuli/speech_phi-45_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi-45_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi-45_diotic.wav" controls></audio> |
| -30 | <audio type="audio/wave" src="../data/stimuli/speech_phi-30_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi-30_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi-30_diotic.wav" controls></audio> |
| -15 | <audio type="audio/wave" src="../data/stimuli/speech_phi-15_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi-15_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi-15_diotic.wav" controls></audio> |
| 0 | <audio type="audio/wave" src="../data/stimuli/speech_phi0_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi0_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi0_diotic.wav" controls></audio> |
| 15 | <audio type="audio/wave" src="../data/stimuli/speech_phi15_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi15_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi15_diotic.wav" controls></audio> |
| 30 | <audio type="audio/wave" src="../data/stimuli/speech_phi30_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi30_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi30_diotic.wav" controls></audio> |
| 45 | <audio type="audio/wave" src="../data/stimuli/speech_phi45_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi45_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi45_diotic.wav" controls></audio> |
| 60 | <audio type="audio/wave" src="../data/stimuli/speech_phi60_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi60_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi60_diotic.wav" controls></audio> |
| 75 | <audio type="audio/wave" src="../data/stimuli/speech_phi75_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi75_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi75_diotic.wav" controls></audio> |
| 90 | <audio type="audio/wave" src="../data/stimuli/speech_phi90_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi90_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi90_diotic.wav" controls></audio> |
| 105 | <audio type="audio/wave" src="../data/stimuli/speech_phi105_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi105_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi105_diotic.wav" controls></audio> |
| 120 | <audio type="audio/wave" src="../data/stimuli/speech_phi120_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi120_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi120_diotic.wav" controls></audio> |
| 135 | <audio type="audio/wave" src="../data/stimuli/speech_phi135_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi135_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi135_diotic.wav" controls></audio> |
| 150 | <audio type="audio/wave" src="../data/stimuli/speech_phi150_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi150_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi150_diotic.wav" controls></audio> |
| 165 | <audio type="audio/wave" src="../data/stimuli/speech_phi165_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi165_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi165_diotic.wav" controls></audio> |
| 180 | <audio type="audio/wave" src="../data/stimuli/speech_phi180_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/castanets_phi180_diotic.wav" controls></audio> | <audio type="audio/wave" src="../data/stimuli/pink_noise_phi180_diotic.wav" controls></audio> |